# Chapter 10 인공 신경망 소개

이전 부분은 교재 참고

### 10.1.3 퍼셉트론

In [0]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:,(2,3)] #꽃잎의 길이와 너비
y = (iris.target==0).astype(np.int) #부채붓꽃(Iris Setosa)인가?

per_clf = Perceptron(random_state=42)
per_clf.fit(X,y)

y_pred = per_clf.predict([[2,0.5]])

사이킷런에서 제공하는 Perceptron 클래스(하나의 TLU 네트워크를 구현함)를 이용했다.

## 10.2 텐서플로의 고수준 API로 다층 퍼셉트론 훈련하기

텐서플로로 다층 퍼셉트론(MLP)을 훈련시키는 가장 간단한 방법은 사이컷런과 호환되는 고수준 API인 TF.Learn을 사용하는 것이다. 

In [2]:
import tensorflow as tf

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100],n_classes=10,feature_columns=feature_cols) 
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train,y_train,batch_size=50,steps=40000)

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa061df28d0>, '_master': '',

SKCompat()

은닉층 2개(각각의 뉴런 수는 300개와 100개)와 10개의 뉴런을 가진 소프트맥스 출력층 하나로 구성된 분류 문제용 심층 신경망(DNN)을 훈련시켰다. 

먼저 훈련 세트로부터 실숫값으로 된 열을 만들고, DNNClassifier의 객체를 만든 뒤 사이킷런 인터페이스와 호환되게 만들어주는 SKCompat 클래스로 감쌌다. 마지막으로 샘플이 50개인 배치로 훈련을 40,000번 반복했다.

In [6]:
from sklearn.metrics import accuracy_score
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test,y_pred['classes'])

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4vgz18qc/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.983

위에서 훈련한 모델이 테스트 세트에서 98.3%의 정확도를 낸다는 것을 확인하였다. (성능이 매우 좋음을 알 수 있음)

## 10.3 텐서플로의 저수준 API로 심층 신경망 훈련하기

네트워크의 구조를 더 상세히 제어하려면 텐서플로의 저수준 파이썬 API가 나을 수 있다. 

### 10.3.1 구성 단계

In [0]:
n_inputs = 28*28 #MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

입력과 출력 크기를 지정하고 은닉층의 뉴런 수를 설정했다.

In [0]:
X = tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y = tf.placeholder(tf.int32, shape=(None),name="y")

9장에서 한 것처럼 플레이스홀더 노드를 사용해 훈련 데이터와 타깃을 표현했다. 

X는 특성의 수가 28*28(픽셀 당 하나의 특성이므로)임을 알고 있지만 훈련 배치에 몇 개의 샘플이 포함될지 모르기 때문에 크기를 일부분만 정의했다. 

y는 값이 샘플당 하나인 1차원 텐서라는 것은 알지만 훈련 배치의 크기를 알 수 없기 때문에 크기를 (None)이라고 정의해주었다.

In [0]:
def neuron_layer(X,n_neurons,name,activation=None):
  with tf.name_scope(name):                                                      
    n_inputs = int(X.get_shape()[1])                                             
    stddev = 2/np.sqrt(n_inputs+n_neurons)                                       
    init = tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)               
    W = tf.Variable(init,name="kernel")                                          
    b = tf.Variable(tf.zeros([n_neurons]),name="bias")                           
    Z = tf.matmul(X,W) + b
    if activation is not None:
      return activation(Z)
    else:
      return Z

2번째 줄: 층 이름으로 이름 범위를 만들었다.

3번째 줄: 입력 행렬의 크기에서 두 번째 차원을 사용해 입력 특성의 수를 구했다.

4~6번째 줄: 가중치 행렬을 담을 W 변수를 만들었다. 이 행렬은 각 입력과 각 뉴런 사이의 모든 연결 가중치를 담고 있는 2D 텐서이다. 그래서 크기는 (n_inputs,n_neurons)가 된다. 표준편차가 2/루트(n_inputs+n_neurons) 인 절단 정규(가우시안) 분포를 사용해 무작위로 초기화된다. 모든 은닉층의 가중치를 무작위로 초기화하는 이유는 대칭성을 피해 경사 하강법 알고리즘이 중단되지 않도록 하기 위함이다.

7번째 줄: 뉴런마다 하나의 편향을 갖도록 변수 b를 만들고 0으로 초기화했다. (여기서는 대칭 문제가 없어서 0으로 초기화해도 문제가 되지 않는다.)

8번째 줄: Z =X·W + b 를 계산하기 위한 그래프를 만들었다. 이는 층에 있는 모든 뉴런과 배치에 있는 모든 샘플에 대해 입력에 대한 가중치 합에 편향을 더하는 계산을 효율적으로 한 번에 수행한다.

9번째 줄~마지막 줄: tf.nn.relu와 같은 activation 매개변수가 지정되어 있으면 activation(Z)(즉 max(0,Z))를 반환하고, 그렇지 않으면 그냥 Z를 반환한다.


이로써 하나의 뉴런 층을 만드는 함수를 만들었다.

In [0]:
with tf.name_scope("dnn"):
  hidden1 = neuron_layer(X,n_hidden1,name="hidden1",activation=tf.nn.relu)
  hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",activation=tf.nn.relu)
  logits = neuron_layer(hidden2, n_outputs, name="outputs")

첫 번째 은닉층은 X를 입력으로 받고, 두 번째는 첫 번째 은닉층의 출력을 입력으로 받는다. 마지막 출력층은 두 번째 은닉층의 출력을 입력으로 받는다.

여기에서도 이름 범위를 사용해 정리했다.

In [13]:
with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X,n_hidden1,name="hidden1",activation=tf.nn.relu)
  hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",activation=tf.nn.relu)
  logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.Dense instead.


위의 코드를 neuron_layer() 함수가 아니라 dense() 함수를 사용하도록 수정해보았다.

In [0]:
with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
  loss = tf.reduce_mean(xentropy,name="loss")

훈련에 사용할 비용 함수를 정의했다. 크로스 엔트로피를 사용했다. 크로스 엔트로피는 모델이 타깃 클래스에 대해 낮은 확률을 추정하지 않도록 제약을 가한다. 

In [0]:
learning_rate = 0.01

with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)

GradientDescentOptimizer를 사용해 비용 함수를 최소화시키도록 모델 파라미터를 조정했다.

In [0]:
with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(logits,y,1)
  accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

먼저 in_top_k() 함수를 사용해 샘플마다 가장 큰 로짓이 타깃 클래스에 해당하는지 여부를 확인해 신경망의 예측이 맞는지 결정했다. 정확도를 사용해 성능을 측정했다.

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

모든 변수를 초기화하는 노드를 만들고 훈련된 모델 파라미터를 디스크에 저장하기 위한 Saver 객체를 생성했다.

여기까지가 구성 단계이다. 입력과 출력을 위한 플레이스 홀더를 만들었고 뉴런의 층을 구성하는 함수를 만들고 이 함수를 사용해 심층 신경망을 생성했다. 비용 함수와 옵티마이저를 생성하고 마지막에 성능 지표를 정의했다. 이 다음은 실행 단계이다.

### 10.3.2 실행단계

In [0]:
n_epochs = 40
batch_size = 50

앞에서 MNIST 데이터셋을 이미 읽어 들여왔으므로 이 단계는 생략하고 실행시킬 에포크 횟수와 미니배치 크기를 정의했다.

In [26]:
with tf.Session() as sess:
  init.run()
  for X_batch,y_batch in shuffle_batch(X_train,y_train,batch_size):
    sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
    acc_batch = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
    acc_valid = accuracy.eval(feed_dict={X:X_valid,y:y_valid})
    print(epoch,"배치 데이터 정확도:",acc_batch,"검증 세트 정확도:",acc_valid)

  save_path = saver.save(sess,"./my_model_final.ckpt")


NameError: ignored

에러

텐서플로 세션을 열고 init 노드를 실행해서 모든 변수를 초기화했다. 그 다음 바깥쪽 훈련 루프를 실행했다. 매 에포크에서 훈련 데이터의 크기를 미니배치 크기로 나눈 횟수만큼 반복했다. 그리고 에포크의 끝에서 마지막 미니배치와 검증 데이터를 사용해 모델을 평가하여 결과를 출력했다. 마지막으로 모델 파라미터를 디스크에 저장했다.

### 10.3.3 신경망 사용하기 

신경망을 훈련시키고 나면 이를 사용해 예측을 만들 수도 있다. 이 때 구성 단계는 그대로 재사용할 수 있지만 실행 단계는 다음과 같이 수정이 필요하다.

In [27]:
with tf.Session() as sess:
  saver.restore(sess,"./my_model_final.ckpt")
  X_new_scaled = [...]
  Z = logits.eval(feed_dict = {X:X_new_scaled})
  y_pred = np.argmax(Z,axis=1)

ValueError: ignored

에러가 떴지만^^,, 일단 코드만 보면

먼저 디스크로부터 모델 파라미터를 읽어들이고, 분류하려는 새 이미지를 읽는다. 훈련 데이터와 같은 특성 스케일로 조정한다. 그런 다음 logits 노드를 평가한다. 모든 클래스에 대한 추정 확률을 알고 싶다면 로짓에 softmax() 함수를 적용하면 되고, 어떤 클래스 하나를 예측하는 것이라면 간단하게 로짓 값이 가장 큰 클래스를 선택하면 된다. (argmax() 함수가 이 작업을 담당함)





10.4는 코드가 없으니 책 참고하기